In [ ]:
%pip install numpy
%pip install scipy
%pip install matplotlib
%pip install neurokit2 
%pip install pandas
%pip install tqdm

In [ ]:
import os
import pandas as pd
import scipy.io
import numpy as np
import neurokit2 as nk
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.signal import butter, filtfilt

def preprocess_signal(signal, original_fs=500, downsample_factor=10):
    """Preprocess the ECG signal by cleaning, downsampling, and filtering using a Butterworth filter."""
    try:
        # Clean the signal
        signal = np.nan_to_num(signal, nan=0.0, posinf=0.0, neginf=0.0)

        # Optionally remove extreme values 
        signal[signal > 10] = 0
        signal[signal < -10] = 0

        # Downsample
        downsampled_signal = signal[::downsample_factor]

        # new sampling rate
        new_fs = original_fs / downsample_factor
        nyquist = 0.5 * new_fs

        # filter cutoffs relative to sampling rate
        low_cutoff = 0.5
        high_cutoff = 20.0
        low = low_cutoff / nyquist
        high = high_cutoff / nyquist

        if high >= 1.0:
            high = 0.99  # safety margin

        # Design Butterworth filter
        b, a = butter(4, [low, high], btype='band')
        filtered_signal = filtfilt(b, a, downsampled_signal)

        return filtered_signal
    except Exception as e:
        print(f"Error preprocessing signal: {e}")
        return signal  # Return the original signal if preprocessing fails

        
def plot_ecg(signal, sampling_rate, title="ECG Signal"):
    """Plotting ECG signal."""
    try:
        time = np.arange(0, len(signal)) / sampling_rate
        plt.figure(figsize=(10, 4))
        plt.plot(time, signal)
        plt.title(title)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Amplitude")
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"Error plotting ECG signal: {e}")

def read_header_info(header_file):
    """Read .hea header file and extract age, sex, and arrhythmia diagnosies """
    try:
        with open(header_file, 'r') as f:
            lines = f.readlines()

        # Extract age, sex, and arrhythmia condition from the header
        age = sex = arrhythmia = None
        for line in lines:
            if line.startswith("#Age:"):
                age = float(line.split(":")[1].strip())
            elif line.startswith("#Sex:"):
                sex = line.split(":")[1].strip()
            elif line.startswith("#Dx:"):
                arrhythmia_codes = line.split(":")[1].strip().split(",")
                arrhythmia = any(code in arrhythmia_codes for code in [
                                "270492004",  # 1st degree AV block (1AVB)
                                "195042002",  # 2nd degree AV block (2AVB)
                                "54016002",   # 2nd degree AV block Type I (2AVB1)
                                "28189009",   # 2nd degree AV block Type II (2AVB2)
                                "27885002",   # 3rd degree AV block (3AVB)
                                "251173003",  # Atrial Bigeminy (ABI)
                                "284470004",  # Atrial Premature Beats (APB)
                                "233917008",  # Atrioventricular Block (AVB)
                                "698252002",  # Intraventricular Block (IVB)
                                "426995002",  # Junctional Escape Beat (JEB)
                                "251164006",  # Junctional Premature Beat (JPT)
                                "164909002",  # Left Bundle Branch Block (LBBB)
                                "59118001",   # Right Bundle Branch Block (RBBB)
                                "74390002",   # Wolff-Parkinson-White Syndrome (WPW)
                                "426177001",  # Sinus Bradycardia (SB)
                                "164889003",  # Atrial Fibrillation (AFIB)
                                "427084000",  # Sinus Tachycardia (ST)
                                "164890007",  # Atrial Flutter (AF)
                                "426761007",  # Supraventricular Tachycardia (SVT)
                                "713422000",  # Atrial Tachycardia (AT)
                                "233896004",  # Atrioventricular Node Reentrant Tachycardia (AVNRT)
                                "233897008",  # Atrioventricular Reentrant Tachycardia (AVRT)
                                "11157007",   # Ventricular Bigeminy (VB)
                                "75532003",   # Ventricular Escape Beat (VEB)
                                "13640000",   # Ventricular Fusion Wave (VFW)
                                "17338001",   # Ventricular Premature Beat (VPB)
                                "195060002",  # Ventricular Preexcitation (VPE)
                                "251180001"   # Ventricular Escape Trigeminy (VET)
                            ])


        return {"Age": age, "Sex": sex, "Arrhythmia": arrhythmia}
    except Exception as e:
        print(f"Error reading header {header_file}: {e}")
        return None

def calculate_heart_features(signal, sampling_rate):
    """Calculate heart-related features using Pan-Tompkins method."""
    try:
        # Detect R-peaks using NeuroKit2's Pan-Tompkins based method
        signals, info = nk.ecg_process(signal, sampling_rate=sampling_rate)
        rpeaks = info['ECG_R_Peaks']

        # Check if there are enough R-peaks
        if len(rpeaks) < 2:
            print(f"Not enough R-peaks detected. Found: {len(rpeaks)}")
            return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ""

        # RR intervals
        rr_intervals = np.diff(rpeaks) / sampling_rate

        if len(rr_intervals) == 0:
            print("No RR intervals calculated.")
            return np.nan, np.nan, np.nan, np.nan, np.nan, np.nan, ""

        # BPM Calculation
        bpm = 60 / np.mean(rr_intervals)

        # RR Interval stats (average, min, max, median)
        average_rr = np.mean(rr_intervals)
        min_rr = np.min(rr_intervals)
        max_rr = np.max(rr_intervals)
        median_rr = np.median(rr_intervals)

        # PT Interval (approximate)
        pt_interval = average_rr

        # Local RR differences (sliding window of 10)
        local_rr = [abs(rr_intervals[i] - rr_intervals[i+10]) for i in range(len(rr_intervals)-10)]

        return bpm, average_rr, min_rr, max_rr, pt_interval, median_rr, ','.join(map(str, local_rr))

    except ZeroDivisionError as e:
        print(f"Error calculating heart features: integer division or modulo by zero")
        return None, None, None, None, None, None, "integer division or modulo by zero"

    except Exception as e:
        print(f"Error calculating heart features: {e}")
        return None, None, None, None, None, None, str(e)

def process_single_ecg(file_path, header_path, sampling_rate=500, downsample_factor=10, plot_signal=False, failed_log_file="failed_ecg_files.log"):
    """Process ECG file by averaging features across all leads."""
    try:
        mat_data = scipy.io.loadmat(file_path)
        ecg_data = mat_data['val']  # Shape: (n_leads, n_samples)

        new_sampling_rate = sampling_rate // downsample_factor

        # Store features for each lead
        feature_list = []

        for lead_index, lead in enumerate(ecg_data):
            preprocessed = preprocess_signal(lead, original_fs=sampling_rate, downsample_factor=downsample_factor)
            bpm, avg_rr, min_rr, max_rr, pt_interval, median_rr, local_rr = calculate_heart_features(preprocessed, new_sampling_rate)

            if bpm is not None:
                feature_list.append([
                    bpm, avg_rr, min_rr, max_rr, pt_interval, median_rr
                ])

                if plot_signal:
                    plot_ecg(preprocessed, new_sampling_rate, title=f"{os.path.basename(file_path)} - Lead {lead_index}")

        if not feature_list:
            with open(failed_log_file, "a") as log:
                log.write(f"Failed to process {file_path} - Reason: No valid R-peaks in any lead\n")

            header_info = read_header_info(header_path)
            return {
                'PersonID': os.path.splitext(os.path.basename(file_path))[0],
                'Age': header_info['Age'],
                'Sex': header_info['Sex'],
                'Arrhythmia': header_info['Arrhythmia'],
                'BPM': np.nan,
                'Average_RR_Interval': np.nan,
                'Min_RR_Interval': np.nan,
                'Max_RR_Interval': np.nan,
                'PT_Interval': np.nan,
                'RR_Median': np.nan,
                'Local_RR_Intervals': np.nan
            }

        # Average the feature values
        averaged_features = np.mean(feature_list, axis=0)

        header_info = read_header_info(header_path)

        return {
            'PersonID': os.path.splitext(os.path.basename(file_path))[0],
            'Age': header_info['Age'],
            'Sex': header_info['Sex'],
            'Arrhythmia': header_info['Arrhythmia'],
            'BPM': averaged_features[0],
            'Average_RR_Interval': averaged_features[1],
            'Min_RR_Interval': averaged_features[2],
            'Max_RR_Interval': averaged_features[3],
            'PT_Interval': averaged_features[4],
            'RR_Median': averaged_features[5],
            'Local_RR_Intervals': np.nan  # Skipped averaging local RR strings
        }

    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        with open(failed_log_file, "a") as log:
            log.write(f"Error processing {file_path}: {e}\n")
        return None


def process_all_ecg_files(base_path, downsample_factor=10, output_csv="ecg_data_summary.csv", plot_signals=False):
    """Recursively process all ECG+Header files"""
    ecg_files = []
    header_files = []

    # Go through directories
    for root, dirs, files in os.walk(base_path):
        for file in files:
            if file.endswith('.mat'):
                ecg_path = os.path.join(root, file)
                header_path = os.path.join(root, file.replace('.mat', '.hea'))
                if os.path.exists(header_path):
                    ecg_files.append(ecg_path)
                    header_files.append(header_path)
                else:
                    print(f"Header missing for {ecg_path}")

    print(f"ECG files found: {len(ecg_files)}")
    print(f"Header files found: {len(header_files)}")

    if not ecg_files:
        print("No ECG files found! Exiting.")
        return pd.DataFrame()

    results = []

    for ecg_path, header_path in tqdm(zip(ecg_files, header_files), total=len(ecg_files), desc="Processing ECG Files"):
        result = process_single_ecg(ecg_path, header_path, downsample_factor=downsample_factor, plot_signal=plot_signals)
        if result:
            results.append(result)
        else:
            print("Skipped a file due to processing failure")

    if results:
        df = pd.DataFrame(results)
        df.to_csv(output_csv, index=False)
        print(f"Data saved to {output_csv}")
        return df
    else:
        print("No valid data processed.")
        return pd.DataFrame()

In [ ]:
def main():
    print("Current working directory:", os.getcwd())

    df = process_all_ecg_files(
        base_path="physionet.org",
        downsample_factor=10,
        output_csv="ecg_data_summary.csv",
        plot_signals=False,
    )
    print(df.head())

if __name__ == "__main__":
    main()